In [ ]:
import os
import time

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy.random as npr

import matplotlib
import matplotlib.animation 
import matplotlib.pyplot as plt
matplotlib.rcParams["animation.embed_limit"] = 1024

import skimage
import skimage.io as sio
import skimage.transform
import fracatal

from fracatal.functional_pt.convolve import ft_convolve
from fracatal.functional_pt.compose import make_gaussian, \
        make_mixed_gaussian, \
        make_kernel_field, \
        make_update_function, \
        make_update_step, \
        make_make_kernel_function

from fracatal.functional_pt.metrics import compute_entropy, \
        compute_frequency_ratio, \
        compute_frequency_entropy
        
from fracatal.scripts import v_stability_sweep, stability_sweep     

import IPython

In [ ]:
# pattern params (physical and simulation)
params = {"orbium_unicaudatus": {"mu_k": [0.5], \
                                 "sigma_k": [0.15],\
                                 "amplitudes_k": [1.0],\
                                 "mu_g": 0.15, \
                                 "sigma_g": 0.017,\
                                 "dt": 0.1,\
                                 "kr": 13,\
                                 "notes": "from https://chakazul.github.io/Lenia/JavaScript/Lenia.html"\
                                },\
        "orbium_unicaudatus_0": {"mu_k": [0.5], \
                                 "sigma_k": [0.15],\
                                 "amplitudes_k": [1.0],\
                                 "mu_g": 0.15, \
                                 "sigma_g": 0.016,\
                                 "dt": 0.1,\
                                 "kr": 13,\
                                 "notes": "from Chan 2019 in Complex Systems 28, e.g. Fig. 5"\
                                },\
         "orbium_bicaudatus": {"mu_k": [0.5], \
                                 "sigma_k": [0.15], \
                                 "amplitudes_k": [1.0],\
                                 "mu_g": 0.15, \
                                 "sigma_g": 0.014,\
                                 "dt": 0.1,\
                                 "kr": 13,\
                                 "notes": "from https://chakazul.github.io/Lenia/JavaScript/Lenia.html"\
                                },\
         "orbium_bicaudatus_ignis": {"mu_k": [0.5], \
                                 "sigma_k": [0.15], \
                                 "amplitudes_k": [1.0],\
                                 "mu_g": 0.12, \
                                 "sigma_g": 0.0113,\
                                 "dt": 0.1,\
                                 "kr": 13,\
                                 "notes": "from https://chakazul.github.io/Lenia/JavaScript/Lenia.html"\
                                },\
         "asymdrop": {"mu_k": [0.5], \
                                 "sigma_k": [0.15], \
                                 "amplitudes_k": [1.0],\
                                 "mu_g": 0.12, \
                                 "sigma_g": 0.005,\
                                 "dt": 0.2,\
                                 "kr": 13,\
                                  "notes": "from Q. Davis 2024 in ALife Proc."\
                                 }
         }

In [ ]:
"""
animation functions
"""

def get_fig(grid, my_cmap="jet", invert=False):
    
    global subplot_0
    
    fig, ax = plt.subplots(1,1, facecolor="black", figsize=(6,6))
    
    if grid.shape[1] == 3:
        subplot_0 = ax.imshow(grid.squeeze().permute(1,2,0))
    else:
        if invert:
            display_grid = 1.0 - plt.get_cmap(my_cmap)(grid.squeeze())
        else:
            display_grid = plt.get_cmap(my_cmap)(grid.squeeze())
        subplot_0 = ax.imshow(display_grid, cmap=my_cmap)

    ax.set_xticklabels("")
    ax.set_yticklabels("")
    
    my_dt = dts[0]
    my_kr = krs[0]
    
    corner_x = .08 #/ (grid_dim*6)
    corner_y = .08# / (grid_dim*6)
    step_y = .08#/ (grid_dim*6)
    
    fig.text(corner_x, corner_y, "$\Delta t =$ " + f"{my_dt:.2e}", fontsize=22, color="w")
    fig.text(corner_x, corner_y + 1 * step_y, "$1/k_r = \Delta r = 1/k_r =$ " + f"{1/my_kr:.2e}", fontsize=22, color="w")
    fig.text(corner_x, corner_y + 2 * step_y, "$\mu_g =$ " + f"{mu_g:.2e}", fontsize=22, color="w")
    fig.text(corner_x, corner_y + 3 * step_y, "$\sigma_g =$ " + f"{sigma_g:.2e}", fontsize=22, color="w")
    
    return fig, ax

def update_frame(ii):
    
    global grid
    global dts
    global krs

    if grid.shape[1] == 3:
        subplot_0.set_array(grid.squeeze().permute(1,2,0))
    else:
        subplot_0.set_array(grid.squeeze())


    #sim_index = int(np.floor(ii // increment_every))
    #print(sim_index, ii)
    my_dt = dts[ii]
    my_kr = krs[ii]
    
    kernel = make_kernel(my_kr)    
    update_step = make_update_step(my_update, kernel, my_dt, clipping_function=clipping_fn, mode=mode)\

    #accumulated_dt = 0.0
    #accumulated_dt += my_dt
    
    corner_x = 8 / (grid_dim*6)
    corner_y = 16 / (grid_dim*6)
    step_y = 16/ (grid_dim*6)
    
    fig.texts[0].set_text("$\Delta t =$ " + f"{my_dt:.2e}")
    fig.texts[1].set_text("$1/k_r = \Delta r = 1/k_r =$ " + f"{1/my_kr:.2e}")
    fig.texts[2].set_text("$\mu_g =$ " + f"{mu_g:.2e}")
    fig.texts[3].set_text("$\sigma_g =$ " + f"{sigma_g:.2e}")
    
    plt.tight_layout()
    
    grid = update_step(grid)
    

In [ ]:
# Gliders in Lenia approximate their mathematical "true selves". 

In [ ]:
pattern_name = "orbium_unicaudatus"
my_cmap = "magma"
save_vid = False
tag = ""

pattern = np.load(os.path.join("..", "patterns", f"{pattern_name}.npy"))
pattern_params = params[pattern_name]

grid_dim = 384
kr = pattern_params["kr"]
k0 = 1 * kr
amplitudes = pattern_params["amplitudes_k"]
means_k = pattern_params["mu_k"]
standard_deviations_k = pattern_params["sigma_k"]
mu_g = pattern_params["mu_g"]
sigma_g = 0.0204 #pattern_params["sigma_g"]

start_dt = 0.1
start_kr = 1 * kr

end_dt = 0.01
end_kr = 120
number_frames = 1024

ddt = (end_dt-start_dt) / (number_frames-1)
dkr = (end_kr-start_kr) / (number_frames-1)

dts = np.arange(start_dt, end_dt+ddt, ddt)
krs = np.arange(start_kr, end_kr+dkr, dkr) 

if pattern_name == "asymdrop":
    mode = 1
    # identity (no clipping)
    clipping_fn = lambda x: x
else:
    mode = 0
    clipping_fn = lambda x: np.clip(x, 0, 1.0)


make_kernel = make_make_kernel_function(amplitudes, means_k, standard_deviations_k, dim=grid_dim-6)
my_update = make_update_function(mu_g, sigma_g, mode=mode)

half_dim = grid_dim // 2 - max(pattern.shape)
grid = torch.zeros(1,1, grid_dim, grid_dim)

grid[:,:,half_dim:half_dim+pattern.shape[-2], half_dim:half_dim+pattern.shape[-1]] = torch.tensor(pattern)

fig, ax = get_fig(grid, my_cmap = my_cmap)
plt.close()

my_interval = 30

#.save(f"{pattern_name}_{tag}.mp4")     

matplotlib.animation.FuncAnimation(fig, update_frame, frames=number_frames, interval=my_interval)\
        .save(f"vid1a_{pattern_name}_{tag}.mp4") if save_vid \
        else IPython.display.HTML(\
        matplotlib.animation.FuncAnimation(fig, update_frame, frames=number_frames, interval=my_interval).to_jshtml())

In [ ]:
start_dt = end_dt * 1.
start_kr = 1 * end_kr

end_dt = start_dt + 1e-6
end_kr = start_kr + 1e-6
number_frames = 1024

ddt = (end_dt-start_dt) / (number_frames-1)
dkr = (end_kr-start_kr) / (number_frames-1)

dts = np.arange(start_dt, end_dt+ddt, ddt)
krs = np.arange(start_kr, end_kr+dkr, dkr) 

fig, ax = get_fig(grid, my_cmap = my_cmap)
plt.close()

my_interval = 3
speedup = 1

matplotlib.animation.FuncAnimation(fig, update_frame, frames=number_frames, interval=my_interval)\
        .save(f"vid1b_{pattern_name}_{tag}.mp4") if save_vid \
        else IPython.display.HTML(\
        matplotlib.animation.FuncAnimation(fig, update_frame, frames=number_frames, interval=my_interval).to_jshtml())

In [ ]:
# Or do they? 

In [ ]:
start_dt = end_dt * 1.
start_kr = 1 * end_kr

end_dt = 0.00001
end_kr = start_kr + 1e-6
number_frames = 2048

ddt = (end_dt-start_dt) / (number_frames-1)
dkr = (end_kr-start_kr) / (number_frames-1)

dts = np.arange(start_dt, end_dt+ddt, ddt)
krs = np.arange(start_kr, end_kr+dkr, dkr) 

fig, ax = get_fig(grid, my_cmap = my_cmap)
plt.close()

my_interval = 3
speedup = 1

matplotlib.animation.FuncAnimation(fig, update_frame, frames=number_frames, interval=my_interval)\
        .save(f"vid1c_{pattern_name}_{tag}.mp4") if save_vid \
        else IPython.display.HTML(\
        matplotlib.animation.FuncAnimation(fig, update_frame, frames=number_frames, interval=my_interval).to_jshtml())